# Export Hospital Level Dashboards

**Purpose:**

1. **Optionally authenticate** with `gcloud` (via user prompt).  
2. **Query BigQuery** for facility names (`faci_name` from `drg-viz.00_datasets.hci`).  
3. **(Optional) Save** these names to a local CSV for debugging.  
4. **Launch Selenium** (using `a_source_chrome_webdriver.py`) to open Power BI in **headless** mode,  
   select each facility in the slicer, and **export** a PDF to `./hospital-reports`.


In [ ]:
%pip install -r requirements.txt

In [ ]:

import os
import re
import csv
import sys
import time
import base64
import subprocess

# BigQuery client library
from google.cloud import bigquery

# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Import the helper function that downloads & returns the ChromeDriver path
from a_source_chrome_webdriver import get_webdriver_path


In [ ]:
# -------------------------------------------------------------------
# Configuration
# -------------------------------------------------------------------
BIGQUERY_QUERY = "SELECT faci_name FROM `drg-viz.00_datasets.hci`"
POWER_BI_URL = (
    "https://app.powerbi.com/view?r=eyJrIjoiMzE2OGUxNDEtMzZmMS00MWJlLTljMzctOWFhZWZmNmQzNmZjIiwidCI6ImM4MzU0YWFmLWVjYzUtNGZmNy05NTkwLWRmYzRmN2MxZjM2MSIsImMiOjEwfQ%3D%3D"
)

XPATH_SLICER_DROPDOWN = "//div[@class='slicer-dropdown-menu' and @aria-label='Facility Name']"
XPATH_POWERBI_IFRAME = "//iframe[contains(@src, 'powerbi')]"

# Optional local CSV debug file
HOSPITALS_CSV = os.path.join(os.getcwd(), "cache/hospitals.csv")

# Output folder for PDFs
REPORTS_DIR = os.path.join(os.getcwd(), "hospital-reports")


In [ ]:
import os
import subprocess
import platform
import webbrowser

def can_open_browser():
    """
    Attempt a naive check for whether we can open a local browser.
    - On Windows or macOS, we assume yes (unless you know otherwise).
    - On Linux, we check if DISPLAY is set.
    """
    system = platform.system()
    if system in ("Windows", "Darwin"):
        return True
    # On Linux, check DISPLAY
    if system == "Linux":
        display = os.environ.get("DISPLAY")
        return bool(display)  # True if DISPLAY is not empty
    return False

def authenticate_gcloud():
    """
    If we can open a local browser, run 'gcloud auth application-default login'.
    Otherwise, run '--no-launch-browser' and do the link/copy/paste flow.
    """
    if can_open_browser():
        print("Detected a local browser environment. Attempting normal gcloud login...")
        # Just run gcloud normally
        proc = subprocess.run(
            ["gcloud", "auth", "application-default", "login"],
            capture_output=True,
            text=True
        )
        if proc.returncode == 0:
            print("gcloud authentication successful (local browser).")
        else:
            print("gcloud auth failed, fallback to no-browser approach.\n")
            _no_browser_auth()
    else:
        print("No local browser detected, using no-browser approach.")
        _no_browser_auth()

def _no_browser_auth():
    """
    The fallback approach that:
    - Runs gcloud with --no-launch-browser
    - Captures link from stdout
    - Prompts user for the verification code
    """
    print("Attempting gcloud authentication with no browser...")

    proc = subprocess.Popen(
        ["gcloud", "auth", "application-default", "login", "--no-launch-browser"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        stdin=subprocess.PIPE,
        text=True
    )

    link_found = False
    link = None

    for line in proc.stdout:
        if "Go to the following link" in line:
            link_found = True
            continue

        if link_found and line.strip().startswith("http"):
            link = line.strip()
            link_found = False

            # Print the link
            print(f"\nOpen this link in your browser:\n{link}\n")
            input("Press Enter once you've opened the link and logged in... ")

            # Prompt for verification code
            verification_code = input("Paste the verification code here: ").strip()
            proc.stdin.write(verification_code + "\n")
            proc.stdin.flush()
            continue

        # Optional: print the rest of gcloud output if desired
        # print(line, end="")

    proc.wait()
    if proc.returncode == 0:
        print("gcloud authentication successful (no-browser).")
    else:
        print(f"Error: gcloud authentication failed with return code {proc.returncode}")


In [ ]:
def main():
    """
    Main steps:
      1) Prompt user to authenticate with gcloud (optional).
      2) Query BigQuery for facility names.
      3) Ask user if they want to export PDFs. If yes:
         - Download ChromeDriver
         - Open Power BI (headless)
         - For each facility, select the slicer item & save PDF
    """

    # ----------------------------------------------------------------
    # 1) Prompt user about gcloud auth
    # ----------------------------------------------------------------
    auth_choice = input("Do you want to authenticate with gcloud now? (y/n) ").strip().lower()
    if auth_choice.startswith("y"):
        authenticate_gcloud()
    else:
        print("Skipping gcloud authentication step...")

    # ----------------------------------------------------------------
    # 2) Query BigQuery for facility names
    # ----------------------------------------------------------------
    print(f"\nQuerying BigQuery:\n  {BIGQUERY_QUERY}")
    try:
        client = bigquery.Client()
        query_job = client.query(BIGQUERY_QUERY)
        results = query_job.result()  # Wait for job to complete
    except Exception as e:
        print(f"Error querying BigQuery: {e}")
        sys.exit(1)

    hospital_names = []
    for row in results:
        hospital_names.append(row.faci_name.strip())

    print(f"Retrieved {len(hospital_names)} facility names from BigQuery.")

    # (Optional) Save to CSV
    try:
        os.makedirs(os.path.dirname(HOSPITALS_CSV), exist_ok=True)
        with open(HOSPITALS_CSV, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["facility_name"])
            for name in hospital_names:
                writer.writerow([name])
        print(f"Saved facility names to '{HOSPITALS_CSV}' for debugging.")
    except Exception as e:
        print(f"Warning: Could not save CSV '{HOSPITALS_CSV}': {e}")

    # ----------------------------------------------------------------
    # 3) Ask user if they want to export PDFs now
    # ----------------------------------------------------------------
    export_choice = input(f"\nDo you want to export PDFs for these {len(hospital_names)} hospitals? (y/n) ").strip().lower()
    if not export_choice.startswith("y"):
        print("User chose not to export PDFs. Exiting.")
        return

    # ----------------------------------------------------------------
    # 4) Export PDFs with Selenium (headless)
    # ----------------------------------------------------------------
    print("\nStarting PDF export...")

    # 4a) Download the appropriate ChromeDriver
    try:
        driver_path = get_webdriver_path()
    except Exception as e:
        print(f"Error obtaining ChromeDriver path: {e}")
        sys.exit(1)

    # 4b) Launch Selenium in headless mode
    options = Options()
    options.add_argument("--headless")       # run without GUI
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")    # often helps stability in headless mode

    service = Service(driver_path)
    try:
        driver = webdriver.Chrome(service=service, options=options)
    except Exception as e:
        print(f"Error launching Chrome: {e}")
        sys.exit(1)

    try:
        print(f"Navigating to Power BI report: {POWER_BI_URL}")
        driver.get(POWER_BI_URL)

        # If there's an iframe, switch to it
        try:
            iframe = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, XPATH_POWERBI_IFRAME))
            )
            driver.switch_to.frame(iframe)
            print("Switched to the Power BI iframe.")
        except TimeoutException:
            print("No iframe found; continuing without frame switch...")

        # Locate the "Facility Name" slicer
        try:
            slicer_dropdown = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, XPATH_SLICER_DROPDOWN))
            )
            print("Found the slicer dropdown for 'Facility Name'.")
        except TimeoutException:
            print("Error: Timeout - Could not find 'Facility Name' slicer dropdown.")
            driver.quit()
            sys.exit(1)

        # Ensure the PDF output folder exists
        os.makedirs(REPORTS_DIR, exist_ok=True)

        # 4c) Iterate each facility and export
        for hospital in hospital_names:
            print(f"\nSelecting hospital: {hospital}")
            # Click the slicer dropdown
            try:
                slicer_dropdown.click()
                time.sleep(1)
            except Exception as e:
                print(f"Error clicking dropdown for '{hospital}': {e}")
                continue

            # Select the hospital from the dropdown
            try:
                option = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//span[text()='{hospital}']"))
                )
                option.click()
                print(f"Clicked '{hospital}' in the slicer.")
            except TimeoutException:
                print(f"Warning: '{hospital}' not found in the slicer list. Skipping.")
                # close the dropdown
                driver.find_element(By.TAG_NAME, "body").click()
                continue

            # Wait for visuals to update
            time.sleep(5)

            # Export to PDF
            try:
                safe_name = re.sub(r'[\\/*?:"<>|]', "_", hospital)
                pdf_data = driver.execute_cdp_cmd("Page.printToPDF", {"printBackground": True})
                pdf_path = os.path.join(REPORTS_DIR, f"{safe_name}.pdf")
                with open(pdf_path, "wb") as pdf_file:
                    pdf_file.write(base64.b64decode(pdf_data['data']))
                print(f"Saved PDF as '{pdf_path}'.")
            except Exception as e:
                print(f"Error saving PDF for '{hospital}': {e}")

        print("\nPDF export complete. Files saved in:", REPORTS_DIR)

    finally:
        driver.quit()


In [ ]:
if __name__ == "__main__":
    main()